In [3]:
import heapq
import os

In [4]:
class BinaryTreeNode:
    def __init__(self, value, freq) -> None:
        self.value = value 
        self.freq = freq
        self.left = None
        self.right = None

    def __lt__(self, other):
        return self.freq < other.freq

    def __eq__(self, other):
        return self.freq == other.freq

In [24]:
class HuffmanCoding:
    def __init__(self, path) -> None:
        self.path = path 
        self.__heap = []
        self.__codes = {}
        self.__reverseCodes = {}

    def __makeFreqDict(self, txt):
        freqDict = {}

        for char in txt:
            if char not in freqDict:
                freqDict[char] = 0
            freqDict[char] += 1
        
        return freqDict

    def __buildHeap(self, freqDict):
        for key in freqDict:
            freq = freqDict[key]
            node = BinaryTreeNode(key, freq)
            heapq.heappush(self.__heap, node)

    def __buildTree(self):
        while len(self.__heap) > 1:
            node1 = heapq.heappop(self.__heap)
            node2 = heapq.heappop(self.__heap)

            freqSum = node1.freq + node2.freq

            newNode = BinaryTreeNode(None, freqSum)
            newNode.left = node1
            newNode.right = node2

            heapq.heappush(self.__heap, newNode)
        
        return
    
    def __buildCodes(self):
        if len(self.__heap) == 0:
            return
        
        root = heapq.heappop(self.__heap)
        self.__buildCodesHelper(root, '')
    
    def __buildCodesHelper(self, root, currBits):
        if root is None:
            return

        if root.value is not None:
            self.__codes[root.value] = currBits
            self.__reverseCodes[currBits] = root.value

        self.__buildCodesHelper(root.left, currBits + '0')
        self.__buildCodesHelper(root.right, currBits + '1')

    def __getEncodedTxt(self, txt):
        encodedTxt = ''

        for char in txt:
            encodedTxt += self.__codes[char]
        
        return encodedTxt
    
    def __getPaddedEncodedTxt(self, encodedTxt):
        paddedAmt = 8 - len(encodedTxt) % 8
        if paddedAmt == 8:
            paddedAmt = 0

        paddedEncodedTxt = encodedTxt + '0' * paddedAmt
        paddedInfo = '{0:08b}'.format(paddedAmt)
        paddedEncodedTxt = paddedInfo + paddedEncodedTxt

        return paddedEncodedTxt
    
    def __getByteArr(self, paddedEncodedTxt):
        arr = []

        for i in range(0, len(paddedEncodedTxt), 8):
            byte = paddedEncodedTxt[i : i + 8]
            arr.append(int(byte, 2))
        
        return arr

    def compress(self):
        file_name, file_extension = os.path.splitext(self.path)
        output_path = file_name + ".bin"
        with open(self.path, 'r', encoding='utf-8') as file, open(output_path, 'wb') as output:
            # Make frequency dictionary using the text
            text = file.read()
            text = text.rstrip()
            if not text:
                print("The file is empty.")
                return None

            freq_dict = self.__makeFreqDict(text)

            # Construct the heap from the frequency_dict
            self.__buildHeap(freq_dict)

            # Construct the binary tree from the heap
            self.__buildTree()
            self.__buildCodes()

            # Creating the encoded text using the codes
            encoded_text = self.__getEncodedTxt(text)

            # Put this encoded text into the binary file
            padded_encoded_text = self.__getPaddedEncodedTxt(encoded_text)
            bytes_array = self.__getByteArr(padded_encoded_text)

            # Write the binary data to the output file
            final_bytes = bytes(bytes_array)
            output.write(final_bytes)
        
        print('Compressed !!!')
        return output_path
    
    def __removePadding(self,text):
        padded_info = text[ : 8]
        extra_padding = int(padded_info, 2)
        text = text[8 : ]
        text_after_padding_removed = text[ : -1 * extra_padding]
        return text_after_padding_removed
    
    def __decodeText(self,text):
        decoded_text = ''
        current_bits = ''

        for bit in text:
            current_bits+=bit
            if current_bits in self.__reverseCodes:
                character=self.__reverseCodes[current_bits]
                decoded_text += character
                current_bits = ''

        return decoded_text
                
    def decompress(self,input_path):
        filename, file_extension = os.path.splitext(self.path)
        output_path = filename + "_decompressed" + ".txt"
        
        with open(input_path,'rb') as file,open(output_path,'w')as output:
            bit_string = ''
            byte = file.read(1)
            while byte:
                byte = ord(byte)
                bits = bin(byte)[2:].rjust(8,'0')
                bit_string += bits
                byte = file.read(1)
            actual_text=self.__removePadding(bit_string)
            decompressedTxt = self.__decodeText(actual_text)
            output.write(decompressedTxt)   
            print('Decompressed !!!')

In [25]:
path='/home/chandershekhar/Documents/PythonWithDsa/DSA/12.HuffmanCoding/hello.txt'
h=HuffmanCoding(path)
output_path=h.compress()
h.decompress(output_path)    

Compressed !!!
Decompressed !!!
